In [ ]:
##### Set up Plummer DF to place the center of the DF at some radius #####

# mwp = MWPotential2014

b_param = 1.07*((10**-3)/ro) # 1.07 is in pc and we have to get it in natural units so convert pc to kpc, then divide by ro to get b in natural units

print ('This is b_param in natural units:',b_param)
# b_param = 1.

##### Set the amplitude of the cluster potential #####

amp = 10**-7 # look at July 29, 2022 notes on iPad this is the order of magnitude from mass fraction

# amp = (10**4)*u.M_sun
########################################################

##### Set up the cluster potential which is made from the Plummer Potential and the Plummer DF #####

# pp = potential.PlummerPotential(amp = 1, b = b_param, ro=None, vo=None) # need to change amp and b param. b*ro so need to check out 6791 and see how big it is for scale parameter (1 kpc or 100 kpc)
pp = PlummerPotential(amp = amp, b = b_param, ro=None, vo=None) # need to change amp and b param. b*ro so need to check out 6791 and see how big it is for scale parameter (1 kpc or 100 kpc)
# b was found doing a volumn integration with the Plummer density and using r=r_{h}=2.3kpc from Martinez-Medina+18
# look at onenote notes from September 29, 2021

pdf = isotropicPlummerdf(pot=pp, ro=None, vo=None)
###########################################################

##### Set up the particles in the sample #####

sam = pdf.sample(n=nbatch*nsample, return_orbit=True) # Output of sample is of the form: (R, vR, Vt, z, vz, phi)

############################################################

##### Place cluster at bar's corotation #####

x0,v0= [0.,L5,0.], [vcirc(mwp,R = L5),0.,0.] #Put cluster at L5 neg velocity makes cluster rotate ccw

######################################################################

##### Shift position of the cluster and convert to cylindrical coordinates #####

shifted_pos = coords.rect_to_cyl(sam.x()+x0[0],sam.y()+x0[1],sam.z()+x0[2])

######################################################################

##### Shift velocity of the cluster and convert to cylindrical coordinates #####

shifted_vel= coords.rect_to_cyl_vec(sam.vx()+v0[0],sam.vy()+v0[1],sam.vz()+v0[2],shifted_pos[0],shifted_pos[1],shifted_pos[2],cyl=True) #True: position is already in cyl coord

########################################################################

##### Set up cluster's orbits #####

shifted_cluster = Orbit(np.array([shifted_pos[0],shifted_vel[0],shifted_vel[1],shifted_pos[2],shifted_vel[2],shifted_pos[1]]).T)

# The shifted_cluster output is in the form: (R, vr, phi,, vphi, z, vz)
##########################################################################

##### Wrap the cluster so that the Plummer Potential itself is shifted to the L4/5 using RotateAndTiltWrapper #####

'''
https://docs.galpy.org/en/latest/reference/potentialrotateandtiltwrapper.html#galpy.potential.RotateAndTiltWrapperPotential
'''

rtwp = RotateAndTiltWrapperPotential(amp=1.0, galaxy_pa=None, zvec=None, offset=[0,L5,0], pot=pp)

##### and make sure it's rotating with the bar using SolidBodyRotationWrapper #####

'''
https://docs.galpy.org/en/latest/reference/potentialsolidbodyrotationwrapper.html?highlight=wrapper

amp - amplitude to be applied to the potential (default: 1.)

pot - Potential instance or list thereof; this potential is made to rotate around the z axis by the wrapper
omega= (1.) the pattern speed (can be a Quantity)

pa= (0.) the position angle (can be a Quantity) # phi -> phi + omega*t + pa, so we want pa = 0 
'''

sbwp = SolidBodyRotationWrapperPotential(amp=1.0, pot=rtwp, omega=omegabo, pa=0)
##################################################################
